In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
ls

anchor_explanation_cache_checkpoint.pkl
anchor_explanations_coverage.pkl
auxiliary_mushrooms.py
best_model_seq_contrastive.keras
dice_explanation_cache_checkpoint.pkl
dice_explanation_cache_checkpoint_test.pkl
dice_explanations_coverage.pkl
dummy_auc_palm_scale_100_window1.pkl
dummy_auc_palm_scale_100_window2.pkl
dummy_auc_palm_scale_100_window3.pkl
dummy_auc_palm_scale_100_window4.pkl
dummy_auc_palm_scale_100_window5.pkl
dummy_auc_palm_scale_100_window6.pkl
dummy_auc_palm_scale_100_window7.pkl
dummy_prop_perturbation_scale_10_acc.pkl
dummy_prop_perturbation_scale_10_acc_test_fin.pkl
dummy_prop_perturbation_scale_10_acc_train_fin.pkl
extracted_data/
fast/
formatted_table_image.png
lime_auc_palm_scale_100_w0.pkl
lime_auc_palm_scale_100_window1.pkl
lime_auc_palm_scale_100_window2.pkl
lime_auc_palm_scale_100_window3.pkl
lime_auc_palm_scale_100_window4.pkl
lime_auc_palm_scale_100_window5.pkl
lime_explanation_cache_checkpoint.pkl
lime_explanations.pkl
lime_explanations_v02.pkl_window1
lime_

In [3]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # 0 - first gpu, 1 - second, "0,1" - both gpu, first used

import numpy as np
import pickle
import tensorflow as tf
import warnings
import math
import shap

from aeon.datasets import load_classification
from numpy import mean, std
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.layers import Dense, Flatten, Dropout, Reshape, ConvLSTM1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.python.framework import ops
from tensorflow.python.keras.initializers import glorot_uniform
from tensorflow.python.util import deprecation
from utils.windowshap import SlidingWindowSHAP

# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.layers import Dense, Flatten, Dropout, Reshape

print(tf.__version__)  # print(tf.keras.__version__)

tf.compat.v1.disable_v2_behavior()

warnings.filterwarnings('ignore')
ops.logging.set_verbosity(ops.logging.ERROR)
tf.get_logger().setLevel('FATAL')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
deprecation._PRINT_DEPRECATION_WARNINGS = False
os.environ['TF_ENABLE_DEPRECATION_WARNINGS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

gpus = tf.config.list_physical_devices("GPU")
print(gpus)
gpu_to_limit = gpus[0]  # or 1, watch out for CUDA_VISIBLE_DEVICES
print(gpu_to_limit)

gpu_mem_mb = 24564  # tyle mamy
tf.config.set_logical_device_configuration(
    gpu_to_limit, [tf.config.LogicalDeviceConfiguration(memory_limit=gpu_mem_mb // 4)]
)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

data_path = "shared/UCI-Benchmark/"
batch_size = 64

2024-11-06 13:57:56.280919: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 13:57:57.158845: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


2.13.1
Instructions for updating:
non-resource variables are not supported in the long term
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Num GPUs Available:  1


# Real Data example

In [4]:
# convlstm model
def divisorGenerator(n):
    large_divisors = []
    for i in range(1, int(math.sqrt(n) + 1)):
        if n % i == 0:
            yield i
            if i * i != n:
                large_divisors.append(n / i)
    for divisor in reversed(large_divisors):
        yield divisor


def load_dataset_aeon(dsname):
    meta_data = None
    if len(load_classification(dsname)) == 3:
        X, y, meta_data = load_classification(dsname)
    elif len(load_classification(dsname)) == 2:
        print("Warning: no meta_data")
        X, y = load_classification(dsname)
    else:
        raise ValueError("Invalid dataset")

    X = np.moveaxis(X, 1, 2)

    le = LabelEncoder()
    y = le.fit_transform(y)

    # Print to check label transformation
    print("Transformed labels:", y[:10])
    print("Unique classes after encoding:", le.classes_)

    trainX, testX, trainy, testy = train_test_split(X, y)

    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    return trainX, trainy, testX, testy


def generate_label_weights(train_y):
    # Count the occurrences of each class by summing across rows
    class_counts = np.sum(train_y, axis=0)

    # Calculate the total number of samples
    total_samples = np.sum(class_counts)

    # Calculate the weight for each class
    class_weights = {}
    num_classes = train_y.shape[1]
    for i in range(num_classes):
        class_weights[i] = total_samples / (num_classes * class_counts[i])

    return class_weights


def train_lstmconv_model(trainX, trainy, testX, testy, kernel=9):
    # Clear previous session to avoid issues with variable initialization
    tf.keras.backend.clear_session()

    verbose, epochs, batch_size = 0, 25, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]

    dg = divisorGenerator(n_timesteps)
    n_steps = int([next(dg) for i in range(3)][-1])
    n_length = int(n_timesteps / n_steps)

    print(f'Input shape (time, rows, channels): {(n_steps, n_length, n_features)}')
    model = Sequential()
    model.add(Reshape((n_steps, n_length, n_features), input_shape=(n_timesteps, n_features)))
    # model.add(Reshape((n_steps, n_length, n_features), input_shape=trainX.shape[1:]))
    model.add(ConvLSTM1D(filters=64, kernel_size=kernel, padding='same', activation='relu', strides=1,
                         data_format='channels_last',
                         return_sequences=True,
                         input_shape=(n_steps, n_length, n_features))
              )
    model.add(ConvLSTM1D(filters=32, kernel_size=kernel, padding='same', activation='relu', strides=1,
                         data_format='channels_last',
                         return_sequences=True,
                         input_shape=(n_steps, n_length, n_features))
              )
    model.add(Dropout(0.5))
    model.add(Flatten(name='embedding'))  #is this a latent representation we could use for clustering?
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Build the model explicitly before showing summary (not always necessary)
    model.build((None, n_timesteps, n_features))
    print(model.summary())

    # fit network
    weights = generate_label_weights(trainy)
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, class_weight=weights)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy, model


# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))


# run an experiment
def evaluate_lstmconv_model(dataset, repeats=1, trainX=None, trainy=None, testX=None, testy=None):
    # load data
    if trainX is None or trainy is None or testX is None or testy is None:
        trainX, trainy, testX, testy = load_dataset_aeon(dataset)

    #fill missing with zeros
    trainX = np.nan_to_num(trainX, nan=0)
    testX = np.nan_to_num(testX, nan=0)

    scaler = StandardScaler()
    trainX = scaler.fit_transform(trainX.reshape(-1, trainX.shape[-1])).reshape(trainX.shape)
    testX = scaler.transform(testX.reshape(-1, testX.shape[-1])).reshape(testX.shape)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score, model = train_lstmconv_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r + 1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)
    return model, trainX, trainy, testX, testy, score


In [5]:
def save_bundle(model, trainX, trainy, testX, testy, svtr, svts, dsname,
                dir='./results', explain_prefix="sv", only_explain=False):
    if not os.path.isdir(f'{dir}/{dsname}'):
        os.makedirs(f'{dir}/{dsname}')

    if not only_explain:
        # model.save(f'{dir}/{dsname}/model.h5', save_format='h5')
        model.save(f'{dir}/{dsname}/model_tf', save_format='tf')

        pickle.dump(trainX, open(f'{dir}/{dsname}/trainX.pickle', 'wb'))
        pickle.dump(trainy, open(f'{dir}/{dsname}/trainy.pickle', 'wb'))
        pickle.dump(testX, open(f'{dir}/{dsname}/testX.pickle', 'wb'))
        pickle.dump(testy, open(f'{dir}/{dsname}/testy.pickle', 'wb'))

    pickle.dump(svtr, open(f'{dir}/{dsname}/{explain_prefix}tr.pickle', 'wb'))
    pickle.dump(svts, open(f'{dir}/{dsname}/{explain_prefix}ts.pickle', 'wb'))


def load_bundle(dsname, dir='./results', explain_prefix="sv", only_explain=False):
    model, trainX, trainy, testX, testy = None, None, None, None, None,
    if not only_explain:
        tf_model_path = f'{dir}/{dsname}/model_tf'
        h5_model_path = f'{dir}/{dsname}/model.h5'

        if os.path.exists(tf_model_path):
            print("Loading model in tf format...")
            model = tf.keras.models.load_model(tf_model_path)
        elif os.path.exists(h5_model_path):
            print("Loading model in h5 format...")
            model = tf.keras.models.load_model(h5_model_path,
                                               custom_objects={'GlorotUniform': glorot_uniform,
                                                               'ConvLSTM1D': ConvLSTM1D})
        else:
            raise FileNotFoundError(
                f'Nie znaleziono modelu w formatach SavedModel ani H5 dla {dsname} w katalogu {dir}.')
        # print(model.summary())

        trainX = pickle.load(open(f'{dir}/{dsname}/trainX.pickle', 'rb'))
        trainy = pickle.load(open(f'{dir}/{dsname}/trainy.pickle', 'rb'))
        testX = pickle.load(open(f'{dir}/{dsname}/testX.pickle', 'rb'))
        testy = pickle.load(open(f'{dir}/{dsname}/testy.pickle', 'rb'))

    svtr = pickle.load(open(f'{dir}/{dsname}/{explain_prefix}tr.pickle', 'rb'))
    svts = pickle.load(open(f'{dir}/{dsname}/{explain_prefix}ts.pickle', 'rb'))

    return model, trainX, trainy, testX, testy, svtr, svts

# Load and test multivariate/EthanolConcentration

In [6]:
model_to_fix = "EthanolConcentration"

In [6]:
model, trainXo, trainyo, testX, testy, trainsvo, testsv = load_bundle(model_to_fix, dir=data_path + 'ds/multivariate')

model

2024-11-04 18:44:55.690744: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6141 MB memory:  -> device: 0, name: NVIDIA RTX A5500, pci bus id: 0000:9c:00.0, compute capability: 8.6
2024-11-04 18:44:55.703633: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-11-04 18:44:55.820451: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1/bias/Assign' id:688 op device:{requested: '', assigned: ''} def:{{{node dense_1/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_1/bias, dense_1/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-11-04 18:44:56.076527: W tensorflow/c/c_api.cc:304] Operation '{name:'conv_

In [7]:
n_timesteps, n_features = trainXo.shape[1], trainXo.shape[2]
print(n_timesteps, n_features)

n_timesteps, n_features = testX.shape[1], testX.shape[2]
print(n_timesteps, n_features)

1751 3
1751 3


In [8]:
trainX_reshaped = trainXo.reshape(-1, n_timesteps * n_features)
_index = -1
prediction = model.predict(trainX_reshaped[_index].reshape(-1, n_timesteps, n_features))
prediction

2024-11-04 19:17:52.029082: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1/Softmax' id:694 op device:{requested: '', assigned: ''} def:{{{node dense_1/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_1/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-11-04 19:17:52.168667: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-11-04 19:17:52.277080: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-04 19:17:52.278549: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-04 19:17:52.278585: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:109] Couldn't get ptxas version : FAILED_PRECONDIT

array([[nan, nan, nan, nan]], dtype=float32)

In [9]:
trainsvo[0]

array([[nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan],
       ...,
       [nan, nan, nan],
       [nan, nan, nan],
       [nan, nan, nan]])

In [20]:
trainX_reshaped[:2, :10]

array([[-0.75419424, -0.76078121, -0.76012238, -0.75452964, -0.76086456,
        -0.75996785, -0.75480755, -0.76081826, -0.7601042 , -0.75480755],
       [-0.91855807, -0.91569179, -0.91378102, -0.91907277, -0.91590387,
        -0.91372466, -0.91888545, -0.91611195, -0.91368321, -0.91835152]])

In [32]:
type(load_classification(model_to_fix))
len(load_classification(model_to_fix))
X, y = load_classification(model_to_fix)
X.shape

(524, 3, 1751)

In [31]:
y.size

524

In [40]:
y[:10]

array(['e35', 'e35', 'e35', 'e35', 'e35', 'e35', 'e35', 'e35', 'e35',
       'e35'], dtype='<U3')

# Multivariate model training

In [ ]:
# tf.keras.backend.clear_session()
# df=pd.read_csv('./datasets/summaryMultivariate.csv',header=None,sep='|')
# df = df[0].str.split(',', expand=True).dropna(axis=1)
# df.columns = df.iloc[0]
# df = df[1:]

bg_size = 1000
window_len = 23
stride = 10
shap_version = 'deep'  #'window'
absshap = True  #Wheather to use absolut values for shap, or use sign of shapp in changepoint detection
#limits due to memory limitations of GPUs
SEQUENCE_LENGTH_LIMIT = 2000
DIMENSIONALITY_LIMIT = 300
ACC_LIMIT = 0

directory = 'multivariate'

# skipds = ['CharacterTrajectories',
#  'InsectWingbeat',
#  'JapaneseVowels',
#  'SpokenArabicDigits',
#  'AtrialFibrillation',
#  'Epilepsy',
#  'FaceDetection',
#  'FingerMovements',
#  'HandMovementDirection',
#  'Handwriting',
#  'Libras',
#  'LSST',
#  'PenDigits']

skipds = []
summary = []

for p in [model_to_fix]:  # df['Problem']
    print(f'Time for {p}')
    if p in skipds:
        print(f'Skipping: {p}')
        continue
    # try:
    X, y = load_classification(p)
    X = np.moveaxis(X, 1, 2)
    if X.shape[1] > SEQUENCE_LENGTH_LIMIT:
        print(f'Skipping: {p} due to sequence length')
        summary.append([p, np.nan, 'LENGTH'])
        continue
    if X.shape[2] > DIMENSIONALITY_LIMIT:
        print(f'Skipping: {p} due to dimensionality size')
        summary.append([p, np.nan, 'DIMENSIONALITY'])
        continue

    print(f'Classifying: {p}')

    # _, trainX, trainy, testX, testy, _, _ = load_bundle(p, dir=os.path.join(data_path, 'ds', directory))

    model, trainX, trainy, testX, testy, score = evaluate_lstmconv_model(p)
    if score < ACC_LIMIT:
        print(f'Skipping: {p} due to low score which is {score}')
        skipds.append(p)
        continue
    print(f'Size of the problem: {trainX.shape}')

    if shap_version == 'window':
        #There is a problem with Windowed version for more than two classes
        indexes = np.arange(0, len(trainX))
        np.random.shuffle(indexes)
        maxid = min(bg_size, len(trainX))
        background_data = trainX[indexes[:maxid]]

        sv_ts = np.zeros((len(testX), testX.shape[1], testX.shape[2]))
        sv_tr = np.zeros((len(trainX), trainX.shape[1], trainX.shape[2]))

        for i in range(len(testX)):
            gtw = SlidingWindowSHAP(model, stride, window_len, background_data, testX[i:i + 1], model_type='lstm')
            sv_ts[i, :, :] = gtw.shap_values(num_output=trainy.shape[1])
        for i in range(len(trainX)):
            gtw = SlidingWindowSHAP(model, stride, window_len, background_data, trainX[i:i + 1], model_type='lstm')
            sv_tr[i, :, :] = gtw.shap_values(num_output=trainy.shape[1])
    elif shap_version == 'deep':
        indexes = np.arange(0, len(trainX))
        np.random.shuffle(indexes)
        maxid = min(bg_size, len(trainX))
        background_data = trainX[indexes[:maxid]]

        explainer = shap.DeepExplainer(model, background_data)
        shap_values_ts = explainer.shap_values(testX, check_additivity=False)
        shap_values_tr = explainer.shap_values(trainX, check_additivity=False)
        if absshap:
            sv_ts = abs(np.array(shap_values_ts)).mean(
                axis=0)  # This basically returns the average importance over the feature/sample
            # Not taking into account the sign of shap value, as it is not required
            # for breakpoints calculation
            sv_tr = abs(np.array(shap_values_tr)).mean(
                axis=0)  # This basically returns the average importance over the feature/sample
            # Not taking into account the sign of shap value, as it is not required
            # for breakpoints calculation
        else:
            indexer = np.argmax(model.predict(testX), axis=1)
            sv_ts = []
            for i in range(0, len(testX)):
                sv_ts.append([shap_values_ts[indexer[i]][i, :]])
            sv_ts = np.concatenate(sv_ts)
            indexer = np.argmax(model.predict(trainX), axis=1)
            sv_tr = []
            for i in range(0, len(trainX)):
                sv_tr.append([shap_values_tr[indexer[i]][i, :]])
            sv_tr = np.concatenate(sv_tr)

    save_bundle(model, trainX, trainy, testX, testy, sv_tr, sv_ts, p, dir=os.path.join(data_path, 'ds', directory))
    summary.append([p, score, 'OK'])

    # except Exception as e:
    #     summary.append([p, np.nan, str(e)])
    #     print(f'Failed...')
    #     skipds.append(p)

    tf.keras.backend.clear_session()

# multi_model_summary = './results/multivariate-new/summary.csv'
# pd.DataFrame(summary,columns=['dataset','score','status']).to_csv(multi_model_summary,index=False)


Time for EthanolConcentration
Classifying: EthanolConcentration
Transformed labels: [0 0 0 0 0 0 0 0 0 0]
Unique classes after encoding: ['e35' 'e38' 'e40' 'e45']
Input shape (time, rows, channels): (103, 17, 3)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 103, 17, 3)        0         
                                                                 
 conv_lstm1d (ConvLSTM1D)    (None, 103, 17, 64)       154624    
                                                                 
 conv_lstm1d_1 (ConvLSTM1D)  (None, 103, 17, 32)       110720    
                                                                 
 dropout (Dropout)           (None, 103, 17, 32)       0         
                                                                 
 embedding (Flatten)         (None, 56032)             0         
                                          

2024-11-04 20:47:41.006624: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12282 MB memory:  -> device: 0, name: NVIDIA RTX A5500, pci bus id: 0000:51:00.0, compute capability: 8.6
2024-11-04 20:47:41.037249: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2024-11-04 20:47:41.098073: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:753] failed to allocate 11.99GiB (12878610432 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2024-11-04 20:47:41.246021: W tensorflow/c/c_api.cc:304] Operation '{name:'training/Adam/conv_lstm1d_1/kernel/v/Assign' id:2704 op device:{requested: '', assigned: ''} def:{{{node training/Adam/conv_lstm1d_1/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/conv_lstm1d_1/kernel/v, training/Adam/conv_lstm1d_1/kernel/v/Initializer/zeros)

>#1: 19.084
[19.083969295024872]
Accuracy: 19.084% (+/-0.000)
Size of the problem: (393, 1751, 3)


In [8]:
summary

[['EthanolConcentration', 19.083969295024872, 'OK']]

In [47]:
tf.compat.v1.initialize_all_variables()

In [10]:
model_new, trainX_new, trainy_new, testX_new, testy_new, trainsv_new, testsv_new = load_bundle(model_to_fix,
                                                                                               dir=data_path + 'ds/multivariate')

model_new

Loading model in tf format...


2024-11-05 08:35:35.123755: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12282 MB memory:  -> device: 0, name: NVIDIA RTX A5500, pci bus id: 0000:51:00.0, compute capability: 8.6
2024-11-05 08:35:35.198756: W tensorflow/c/c_api.cc:304] Operation '{name:'AssignVariableOp_36' id:329 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_36}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](conv_lstm1d_1/bias/v, Identity_36)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [11]:
n_timesteps_new, n_features_new = trainX_new.shape[1], trainX_new.shape[2]
print(n_timesteps_new, n_features_new)

n_timesteps_new, n_features_new = testX_new.shape[1], testX_new.shape[2]
print(n_timesteps_new, n_features_new)

trainX_reshaped_new = trainX_new.reshape(-1, n_timesteps_new * n_features_new)
trainX_reshaped_new[:2, :10]

1751 3
1751 3


array([[-0.91144945, -0.91760858, -0.91162117, -0.9119377 , -0.91810254,
        -0.91160955, -0.91198368, -0.91808251, -0.91130938, -0.91170224],
       [-0.74786201, -0.75629294, -0.75323544, -0.74813022, -0.75630243,
        -0.75365401, -0.74815896, -0.75629294, -0.75386795, -0.74784285]])

In [18]:
_index = -1
prediction_new = model_new.predict(trainX_reshaped_new[_index].reshape(-1, n_timesteps_new, n_features_new))
prediction_new

array([[nan, nan, nan, nan]], dtype=float32)

# Check LIME explanations

In [7]:
model_multi, trainX_multi, trainy_multi, testX_multi, testy_multi, trainlv_multi, testlv_multi = load_bundle(
    "SelfRegulationSCP1", dir=data_path + 'ds/multivariate', explain_prefix="lv")
model_uni, trainX_uni, trainy_uni, testX_uni, testy_uni, trainlv_uni, testlv_uni = load_bundle(
    "InsectWingbeatSound", dir=data_path + 'ds/univariate', explain_prefix="lv")

Loading model in h5 format...


2024-11-06 13:58:50.733399: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_2/bias/Assign' id:1709 op device:{requested: '', assigned: ''} def:{{{node dense_2/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_2/bias, dense_2/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2024-11-06 13:58:50.971119: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_1_1/bias/v/Assign' id:2001 op device:{requested: '', assigned: ''} def:{{{node dense_1_1/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_1_1/bias/v, dense_1_1/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in 

In [10]:
print("X[train]", trainX_multi.shape, "X[test]", testX_multi.shape, "y[train]", trainy_multi.shape, "y[test]",
      testy_multi.shape, "LIME[train]", trainlv_multi.shape, "LIME[test]", testlv_multi.shape)

X[train] (420, 896, 6) X[test] (141, 896, 6) y[train] (420, 2) y[test] (141, 2) LIME[train] (420, 896, 6) LIME[test] (141, 896, 6)


In [11]:
print("X[train]", trainX_uni.shape, "X[test]", testX_uni.shape, "y[train]", trainy_uni.shape, "y[test]",
      testy_uni.shape, "LIME[train]", trainlv_uni.shape, "LIME[test]", testlv_uni.shape)

X[train] (1650, 256, 1) X[test] (550, 256, 1) y[train] (1650, 11) y[test] (550, 11) LIME[train] (1650, 256, 1) LIME[test] (550, 256, 1)
